#### Import Libraries and Declare Variables

In [5]:
import pandas as pd
pd.options.display.max_rows = 50
pd.options.display.max_columns = 50
pd.set_option('float_format', '{:f}'.format)

import dateutil.relativedelta as relativedelta
import datetime
import os
import numpy as np

user = os.getenv('username')

main_folder = "D:/Users/"+user+"/Desktop/Airtel/Source Files/"
report_folder = "D:/Users/"+user+"/Desktop/Airtel/Reports/Federal CLS DPD/Recon/"

curr_mon = datetime.datetime.today().strftime('%b%y') #'01Feb18'
curr_date = datetime.datetime.today().strftime('%d%b%y') #'01Feb18'

if not os.path.exists(report_folder+curr_mon):
    os.makedirs(report_folder+curr_mon)

#### Import Source Files

In [6]:
cls_emi = pd.read_csv(main_folder+'CLS_EMI_Data.csv')
federal_mapping = pd.read_csv(main_folder + 'Federal ID Mapping.csv')
federal_ledger = pd.read_excel(main_folder + 'ADF+LS dump.xlsx') #'ADF-LS-dump.xlsx')

#### Data Recon Federal and CLS

In [8]:
cls_emi_fed = cls_emi[cls_emi['Lender: Account Name'] == 'Federal Bank']

fed_map_cls = federal_mapping[['Loan App Reference Number','Lender Loan_Ref_no']]

# federal_ledger = federal_ledger[(federal_ledger['SCHM_CODE'] == 78228) & (federal_ledger['DISB_DATE'] != '1999-12-31')]
federal_ledger = federal_ledger[(federal_ledger['DISB_DATE'] != '1999-12-31')]
fed_led_id = federal_ledger[['ACCOUNT','SANCTION_REFERENCE_NUMBER']]
fed_led_id = fed_led_id.rename(columns={'SANCTION_REFERENCE_NUMBER':'Loan App Reference Number'})

fed_map_final = pd.merge(fed_map_cls, fed_led_id, on = 'Loan App Reference Number', how = 'outer')

def final_map(loan, l1, l2):
    if l1 == 'nan':
        if l2 == 'nan':
            return 'NA'
        else:
            if loan[:3] != 'adf':
                return 'Ignore'
            else:
                return l2
    else:
        if l2 == 'nan' or l1 == l2:
            return l1
        else:
            if l1[:3] == 'adf':
                return str(l2).replace('.0','')
            else:
                return "Error"
        
fed_map_final['lender_id'] = fed_map_final.apply(lambda x: final_map(str(x['Loan App Reference Number']).lower(),
                                                                     str(x['Lender Loan_Ref_no']).lower(),
                                                                     str(x['ACCOUNT']).lower()), axis=1)

issue_cases = fed_map_final[fed_map_final['lender_id'] == 'Error']
fed_map = fed_map_final[(fed_map_final['lender_id'] != 'Error') & (fed_map_final['lender_id'] != 'Ignore') & (fed_map_final['lender_id'] != 'NA')]

fed_map = fed_map.sort_values('lender_id')
fed_map['lender_id'] = fed_map.apply(lambda x: str(x['lender_id']).upper(), axis=1)
fed_map2 = fed_map[['Loan App Reference Number','lender_id']]

federal_ledger['lender_id'] = federal_ledger.apply(lambda x: str(x['ACCOUNT']).replace('.0',''),axis=1)
merge_data = pd.merge(fed_map2, federal_ledger, on = 'lender_id', how = 'left')
merge_data['ACCOUNT'] = merge_data['ACCOUNT'].fillna('NA')
merge_final = merge_data[merge_data['ACCOUNT'] != 'NA']

cls_emi_col = cls_emi_fed[['Loan App Reference Number','First Monthly Collection Date','Days Past Due wrt MCD',
                           'Delinquent Amount wrt MCD','Monthly Payment','Principal Outstanding']].fillna(0)

merge_final_col = merge_final[['Loan App Reference Number','lender_id','SANCT_LIM','CLR_BAL_AMT','EMI','DPD',
                               'DISB_DATE','INTEREST_ARREARS','OTHER_CHARGES','PENAL_INTEREST_ARREARS',
                               'TOTAL_ARREARS']].fillna(0)

fed_cls_dpd = pd.merge(merge_final_col, cls_emi_col, on = 'Loan App Reference Number', how = 'left')

def first_emi(del_date):
    day_of_month = int(del_date.strftime("%d"))
    if day_of_month < 21:
        first_emi_date = del_date + relativedelta.relativedelta(months=1)
    else:
        first_emi_date = del_date + relativedelta.relativedelta(months=2)
    first_emi_date = first_emi_date.replace(day=5).date()
    return first_emi_date

fed_cls_dpd['Fed_First_EMI_date'] = fed_cls_dpd.apply(lambda x: first_emi(x['DISB_DATE']),axis=1)

def date_conv(date):
    try:
        return datetime.datetime.strptime(date,'%d/%m/%Y').date()
    except Exception as e:
        return date

fed_cls_dpd['First Monthly Collection Date'] = fed_cls_dpd.apply(lambda x: date_conv(x['First Monthly Collection Date']), axis=1)

def preference(fed_dpd, cls_dpd):
    if fed_dpd < cls_dpd:
        return 'Federal'
    else:
        return 'CLS'
    
def final_result(exp, fed, cls):
    if exp == 'Federal':
        return fed
    else:
        return cls
    
fed_cls_dpd['Preference'] = fed_cls_dpd.apply(lambda x: preference(x['DPD'], x['Days Past Due wrt MCD']), axis=1)

fed_cls_dpd['Fed POS'] = fed_cls_dpd.apply(lambda x: max((-1 * x['CLR_BAL_AMT']) - x['INTEREST_ARREARS'] - x['OTHER_CHARGES'] - x['PENAL_INTEREST_ARREARS'],0), axis=1)
fed_cls_dpd['Total_Charges'] = fed_cls_dpd['TOTAL_ARREARS'] - fed_cls_dpd['Delinquent Amount wrt MCD']

fed_cls_dpd['Final DPD'] = fed_cls_dpd.apply(lambda x: final_result(x['Preference'], x['DPD'], x['Days Past Due wrt MCD']), axis=1)
fed_cls_dpd['Final POS'] = fed_cls_dpd.apply(lambda x: final_result(x['Preference'], x['Fed POS'], x['Principal Outstanding']), axis=1)

def explanation(fed_dpd, cls_dpd, fed_first, cls_first, charge):
    if fed_dpd == cls_dpd:
        return 'DPD Match'
    elif fed_dpd < cls_dpd:
        return 'Collections'
    else:
        if str(cls_first).lower() == 'nan':
            return 'Recent Cases'
        elif (fed_first != cls_first) & (charge > 0) & (charge < 200):
            return 'Disbursement Date Issue & Charges Applied'
        elif fed_first != cls_first:
            return 'Disbursement Date Issues'
        elif (charge > 0) & (charge < 200):
            return 'Federal Charges Applied'
        else:
            return 'Please analyze further'
        
fed_cls_dpd['Explanation'] = fed_cls_dpd.apply(lambda x: explanation(x['DPD'], x['Days Past Due wrt MCD'], 
                                                                     x['Fed_First_EMI_date'], 
                                                                     x['First Monthly Collection Date'], 
                                                                     x['Total_Charges']), axis=1)

def dpd_bucket(dpd):
    if dpd == 0:
        return 'Current'
    if dpd <= 30:
        return '1-30'
    if dpd <= 60:
        return '31-60'
    if dpd <= 90:
        return '61-90'
    if dpd > 90:
        return '90+'
    
fed_cls_dpd['Final DPD Bucket'] = fed_cls_dpd.apply(lambda x: dpd_bucket(x['Final DPD']), axis=1)

fed_cls_dpd.to_excel(report_folder + curr_mon + '/Fed Recon Data - ' + curr_mon + '.xlsx', index=False)

fed_cls_dpd['Explanation'].value_counts()

DPD Match                                    8683
Federal Charges Applied                       484
Please analyze further                        217
Collections                                   111
Recent Cases                                   93
Disbursement Date Issue & Charges Applied      48
Disbursement Date Issues                       32
Name: Explanation, dtype: int64

In [9]:
# fed_cls_dpd.to_csv("ram2.csv", index=False)

In [4]:
# def dpd_bucket(dpd):
#     if dpd == 0:
#         return 'Current'
#     if dpd <= 30:
#         return '1-30'
#     if dpd <= 60:
#         return '31-60'
#     if dpd <= 90:
#         return '61-90'
#     if dpd > 90:
#         return '90+'
    
# fed_cls_dpd['Final DPD Bucket'] = fed_cls_dpd.apply(lambda x: dpd_bucket(x['Final DPD']), axis=1)

# fed_cls_dpd.to_excel(report_folder + curr_mon + '/Fed Recon Data - ' + curr_mon + '.xlsx', index=False)
